# Southern Ocean Mode index
annual SST anomaly (50S-35S, 0E-50W), here detrended

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import bll_SOM
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, lag_linregress_3D, xr_quadtrend

In [ ]:
# %%time
# # 2:55
# SOM_GMST_ctrl = SST_index('SOM', 'ctrl', detrend_signal='GMST')
# SOM_GMST_rcp  = SST_index('SOM', 'rcp' , detrend_signal='GMST')
# SOM_GMST_lpi  = SST_index('SOM', 'lpi' , detrend_signal='GMST')
# SOM_GMST_lpd  = SST_index('SOM', 'lpd' , detrend_signal='GMST')
# SOM_GMST_had  = SST_index('SOM', 'had' , detrend_signal='GMST')

# SOM_rgnl_ctrl = SST_index('SOM', 'ctrl', detrend_signal='SOM')
# SOM_rgnl_rcp  = SST_index('SOM', 'rcp' , detrend_signal='SOM')
# SOM_rgnl_lpi  = SST_index('SOM', 'lpi' , detrend_signal='SOM')
# SOM_rgnl_lpd  = SST_index('SOM', 'lpd' , detrend_signal='SOM')
# SOM_rgnl_had  = SST_index('SOM', 'had' , detrend_signal='SOM')

In [ ]:
SOM_GMST_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_ctrl.nc')
SOM_GMST_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_rcp.nc' )
SOM_GMST_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_lpi.nc' )
SOM_GMST_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_lpd.nc' )
SOM_GMST_had  = xr.open_dataarray(f'{path_samoc}/SST/SOM_GMST_dt_raw_had.nc' )

SOM_rgnl_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SOM_raw_ctrl.nc')
SOM_rgnl_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SOM_raw_rcp.nc' )
SOM_rgnl_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SOM_raw_lpi.nc' )
SOM_rgnl_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SOM_raw_lpd.nc' )
SOM_rgnl_had  = xr.open_dataarray(f'{path_samoc}/SST/SOM_SOM_dt_raw_had.nc' )

SOM_rgnl_ctrl -= xr_lintrend( SOM_rgnl_ctrl)
SOM_rgnl_rcp  -= xr_quadtrend(SOM_rgnl_rcp )
SOM_rgnl_lpi  -= xr_lintrend( SOM_rgnl_lpi )
SOM_rgnl_lpd  -= xr_lintrend( SOM_rgnl_lpd )

lowpass(SOM_rgnl_ctrl, 13).to_netcdf(f'{path_samoc}/SST/SOM_ctrl.nc')
lowpass(SOM_rgnl_rcp , 13).to_netcdf(f'{path_samoc}/SST/SOM_rcp.nc' )
lowpass(SOM_rgnl_lpi , 13).to_netcdf(f'{path_samoc}/SST/SOM_lpi.nc' )
lowpass(SOM_rgnl_lpd , 13).to_netcdf(f'{path_samoc}/SST/SOM_lpd.nc' )
lowpass(SOM_rgnl_had , 13).to_netcdf(f'{path_samoc}/SST/SOM_had.nc' )

lowpass(SOM_GMST_ctrl, 13).to_netcdf(f'{path_samoc}/SST/SOM_GMST_dt_ctrl.nc')
lowpass(SOM_GMST_rcp , 13).to_netcdf(f'{path_samoc}/SST/SOM_GMST_dt_rcp.nc' )
lowpass(SOM_GMST_lpi , 13).to_netcdf(f'{path_samoc}/SST/SOM_GMST_dt_lpi.nc' )
lowpass(SOM_GMST_lpd , 13).to_netcdf(f'{path_samoc}/SST/SOM_GMST_dt_lpd.nc' )
lowpass(SOM_GMST_had , 13).to_netcdf(f'{path_samoc}/SST/SOM_GMST_dt_had.nc' )

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5))
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)

L1, = ax.plot(SOM_GMST_ctrl.time/365+1850, lowpass(SOM_GMST_ctrl, 13), c='C0', ls='-', lw=1, label='SST detrended with scaled GMST')
ax.plot(      SOM_GMST_rcp .time/365+ 200, lowpass(SOM_GMST_rcp , 13), c='C1', ls='-', lw=1)
ax.plot(      SOM_GMST_lpd .time/365+1350, lowpass(SOM_GMST_lpd , 13), c='C2', ls='-', lw=1)
ax.plot(      SOM_GMST_lpi .time/365-1600, lowpass(SOM_GMST_lpi , 13), c='C3', ls='-', lw=1)
ax.plot(      SOM_GMST_had .time/365+2350, lowpass(SOM_GMST_had , 13), c='C4', ls='-', lw=1)

L2, = ax.plot(SOM_rgnl_ctrl.time/365+1850, lowpass(SOM_rgnl_ctrl, 13), c='C0', ls='--', lw=1, label='target region regression')
ax.plot(      SOM_rgnl_rcp .time/365+ 200, lowpass(SOM_rgnl_rcp , 13), c='C1', ls='--', lw=1)
ax.plot(      SOM_rgnl_lpd .time/365+1350, lowpass(SOM_rgnl_lpd , 13), c='C2', ls='--', lw=1)
ax.plot(      SOM_rgnl_lpi .time/365-1600, lowpass(SOM_rgnl_lpi , 13), c='C3', ls='--', lw=1)
ax.plot(      SOM_rgnl_had .time/365+2350, lowpass(SOM_rgnl_had , 13), c='C4', ls='--', lw=1)

ax.text(1950, .3, 'CTRL'         , fontsize=16, color='C0')
ax.text(2200, .3, 'RCP'          , fontsize=16, color='C1')
ax.text(1500, .3, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .3, 'pre-ind. low' , fontsize=16, color='C3')
ax.text(2320, .3, 'HadISST'      , fontsize=16, color='C4')

ax.legend(handles=[L1,L2], loc=8, ncol=3, fontsize=14, frameon=False)

ax.set_ylabel('SOM index [K]', fontsize=16)
ax.set_xlabel('time [years]', fontsize=16)
ax.set_xticks(np.arange(0,2700,200))
ax.set_xlim((-50,2550))
f.tight_layout()
f.savefig(f'{path_results}/SST/SOM_index_overview.png')

## spectra
- add red noise spectrum as null hypothesis

- influence of spectrum type `welch`
- 

In [ ]:
SOMs = [SOM_ctrl, SOM_rcp, SOM_lpd, SOM_lpi, SOM_had]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (5):
    label= ['CTRL', 'RCP', 'LPD', 'LPI', 'Had'][i]
    f, Pxx = sp.signal.welch(SOMs[i]-xr_lintrend(SOMs[i]), fs=1)
    ax.loglog(1/f, np.sqrt(Pxx), label=label)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
    
plt.tight_layout()

In [ ]:
SOMs = [SOM_ctrl, SOM_rcp, SOM_lpd, SOM_lpi, SOM_had]

f, ax = plt.subplots(1, 1, figsize=(8,5), sharey=True)
ax.set_ylabel('spectral power', fontsize=16)

for i in range (5):
    label= ['CTRL', 'RCP', 'LPD', 'LPI', 'Had'][i]
    f, Pxx = sp.signal.welch(SOMs[i]-xr_lintrend(SOMs[i]), fs=1)
    ax.semilogy(f, np.sqrt(Pxx), c=f'C{i}', label=label)
    f, Pxx = sp.signal.welch(SOMs[i]-xr_quadtrend(SOMs[i]), fs=1)
    ax.semilogy(f, np.sqrt(Pxx), c=f'C{i}', ls='--')
#     f, Pxx = sp.signal.periodogram(SOMs[i]-xr_lintrend(SOMs[i]), fs=1)
#     ax.semilogy(f, np.sqrt(Pxx), c=f'C{i}', ls='--', lw=.5)

ax.tick_params(labelsize=14)
ax.set_xlabel('period [yr]', fontsize=16)
ax.legend(fontsize=14, frameon=False)
ax.set_ylim(1e-2,1)
    
plt.tight_layout()

## regression plots

In [ ]:
SST_yrly_dt_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)

In [ ]:
len(SST_yrly_dt)

In [ ]:
regr_ctrl = lag_linregress_3D(SOM_cheb13_dt_ctrl, SST_yrly_dt_ctrl)

In [ ]:
from maps import reg